In [2]:
from google.cloud import bigquery
import re
import os
import pandas as pd
#from google.colab import drive
#from google.colab import auth
#drive.mount('/content/drive')
#auth.authenticate_user()
project_id = 'iter-data-storage-pv-uat'
client = bigquery.Client(project_id)

In [3]:
path='Desarrollo/Replicacion UAT/replicacion_prod/sql_input/'

In [12]:
def sqlbigquery():
    with open("replicacion.sql","r") as file:
        statement = file.readlines()
        statement = " ".join(statement)
    return statement

In [15]:
def read_table_bq(query):
  df = pd.read_gbq(query, project_id=project_id, dialect='standard')
  return df

In [7]:
def store_base(df):
  for index, row in df.iterrows():
    nombre_archivo = row['NOMBRE_STORE_PROCEDURE']
    contenido_archivo = row['DDL']
    with open(path + nombre_archivo+'.sql', 'w', newline='') as archivo:
      archivo.write(contenido_archivo)
  return 'Se exporto correctamente los DDL'

# Generar los archivos .sql

In [20]:
store_base(read_table_bq(sqlbigquery()))

C:\Users\jberriot\AppData\Local\Temp\ipykernel_21660\2986615697.py:2: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.read_gbq(query, project_id=project_id, dialect='standard')


'Se exporto correctamente los DDL'

# Ejecutar Jobs por partes

In [21]:
def execute_job(query):
    query_job = client.query(query)
    results = query_job.result()
    for row in results:
        ddl_statement = row['DDL']
        client.query(ddl_statement).result()
    return f'Ejectuado Correctamente'

In [22]:
execute_job(sqlbigquery())

'Ejectuado Correctamente'